AE with one Interaction layers

In [1]:
import numpy as np
import itertools
import os, sys
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

ImportError: No module named matplotlib.pyplot

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from progressBar import ProgressBar
from utils import EarlyStopping

In [ ]:
dnd = []

## Get the dataset

In [ ]:
from torch.utils import data

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, names = 'SM', 
                 template='../data/20190702_20part_PtOrder_v1/{}.npy'):
        if names == 'SM':
            self.SM_names = ['Wlnu', 'qcd', 'ttbar', 'Zll']
            names = self.SM_names
        self.names = names
        
        self.feature_names = ['Pt', 'eta', 'phi', 'charge', 'pId'][:3]
        
        for i, n in enumerate(names):
            ins = np.load(template.format(n)).astype(np.float32)[:,:,:3]
            lbs = i*np.ones(ins.shape[0]).astype(np.int8)
            
            print(n, ':', str(ins.shape[0]))
            
            if i == 0:
                self.inputs = ins
                self.labels = lbs
            else:
                self.inputs = np.concatenate((self.inputs, ins))
                self.labels = np.concatenate((self.labels, lbs))
    
    def __len__(self):
        return self.inputs.shape[0]
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.inputs[idx]

In [ ]:
dataset = {}
# Use to do validation split
# torch.utils.data.random_split(dataset, lengths)
dataset['train'] = Dataset(names=['Wlnu'])

## Define the model

In [ ]:
class AEGraphNet(nn.Module):
    def __init__(self, N_nodes, N_features=5, dim_hidden=20, De=7, Do=None, dim_latent=8, verbose = False):
        super(AEGraphNet, self).__init__()
        self.verbose = verbose
        
        self.p = N_features
        self.No = N_nodes
        self.De = De
        self.Do = 1
        self.Nk = dim_latent
        
        # Define the T/R matrixes
        self.Rr, self.Rs = self.buildEdgesMatrixes() 
        # As soon dim are fixed and connections are the same this is enough
        
        self.encoder_modules = nn.ModuleDict({
            'fr': self.build_dense(dim_in=2*self.p,
                                   dim_out=self.De,
                                   dim_hidden=dim_hidden),
            'fo': self.build_dense(dim_in=self.p + self.De,
                                   dim_out=self.Do,
                                   dim_hidden=dim_hidden),
            'PhiC': self.build_Phi(dim_in=self.No*self.Do,
                                    dim_out=self.Nk)
        })
        
        
        self.decoder_modules = nn.ModuleDict({
            'PhiU': self.build_Phi(dim_in=self.Nk,
                                   dim_out=self.No*self.Do),
            'fr': self.build_dense(dim_in=2*self.Do,
                                   dim_out=self.De,
                                   dim_hidden=dim_hidden),
            'fo': self.build_dense(dim_in=self.Do + self.De,
                                   dim_out=self.p,
                                   dim_hidden=dim_hidden)
        })
        
        
        self.onGPU = False
             
    def buildEdgesMatrixes(self):
        ### Assume fully connected graph
        Ne = self.No * (self.No - 1)
        self.Ne = Ne
        Rr = torch.zeros(self.No, Ne)
        Rs = torch.zeros(self.No, Ne)
        receiver_sender_list = [i for i in itertools.product(range(self.No), range(self.No)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            Rr[r, i] = 1
            Rs[s, i] = 1
        return Rr, Rs
          
    def build_dense(self, dim_in, dim_out, dim_hidden):
        net = nn.Sequential(
                            nn.Linear(dim_in, dim_hidden),
                            nn.ELU(),
                            nn.Linear(dim_hidden, dim_hidden),
                            nn.ELU(),
                            nn.Linear(dim_hidden, dim_out),
                          )
        return net

    def build_Phi(self, dim_in, dim_out):
        Phi = nn.Sequential(
                              nn.Linear(dim_in, int(0.5*(dim_out + dim_in))),
                              nn.ELU(),
                              nn.Linear(int(0.5*(dim_out + dim_in)), dim_out)
                            )
        return Phi
         
    def INlayer(self, x, modules):
        local_p = x.shape[2]
        x = torch.transpose(x, 1, 2).contiguous()

        Orr = torch.matmul(x, self.Rr)
        Ors = torch.matmul(x, self.Rs)
        B = torch.cat([Orr, Ors], dim=1)
        
        ### First MLP ###
        E = modules['fr'](B.view(-1, 2 * local_p))
        del B
        
        E = E.view(-1, self.Ne, self.De)
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = torch.matmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        
        C = torch.cat([x, Ebar], 1)
        del Ebar
        
        C = torch.transpose(C, 1, 2).contiguous()
        O = modules['fo'](C.view(-1, local_p + self.De))
        del C
        local_Do = modules['fo'][-1].out_features
        O = torch.relu(O.view(-1, self.No, local_Do))
        return O
        
    def encode(self, x):
        O = self.INlayer(x, self.encoder_modules)
        O = O.view(-1, self.No * self.Do)
        z = self.encoder_modules['PhiC'](O)
        return z
    
    def decode(self, z):
        O = self.decoder_modules['PhiU'](z)
        O = O.view(-1, self.No, self.Do)
        x = self.INlayer(O, self.decoder_modules)
        
        # --- Formatting x ----
        # pt
        pt = 1. + F.elu(x[:,:,0]).view(-1, x.shape[1], 1)
        # eta
        eta = F.hardtanh(x[:,:,1], min_val=-5, max_val=5).view(-1, x.shape[1], 1)
        # phi
        phi = F.hardtanh(x[:,:,2], min_val=-3.14159, max_val=3.14159).view(-1, x.shape[1], 1)
        # charge
#         x[:,:,3] = F.hardtanh(x[:,:,3], min_val=-1, max_val=2)
        #pId
#         x[:,:,4] = F.hardtanh(x[:,:,4], min_val=0, max_val=6)
        return torch.cat((pt,eta, phi), 2)
        
    
    def forward(self, x):
        z = self.encode(x)
        x_prime = self.decode(z)
        return x_prime
    
    def initWeights(self):
        def weights_init(M):
            if hasattr(M, 'weight'):
                nn.init.xavier_normal_(M.weight.data)
        
        self.apply(weights_init)
    
    def useGPU(self):
        if torch.cuda.is_available():
            print('Current device: {} ({} available)'.format(torch.cuda.current_device(), 
                                                             torch.cuda.device_count()))
            
            for o in self.__dict__.values():
                if o.__class__ == torch.Tensor:
                    o.cuda()
            
            self.onGPU = True
        else: 
            print('GPU not available')

In [ ]:
model = AEGraphNet(
                   N_nodes=dataset['train'].inputs.shape[1],
                   N_features=dataset['train'].inputs.shape[2],
                   dim_hidden=20,
                   De=5,
                   dim_latent=5
                  )

print(model)
trainablePars = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)

model.initWeights()

## Loss

In [ ]:
criterion = nn.L1Loss()

## Training

In [ ]:
training_params = {'batch_size': 10000,
                   'shuffle': True,
                   'num_workers': 0
                  }

max_epochs = 5

In [ ]:
torch.autograd.set_detect_anomaly(True);

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 
                              mode='min',
                              factor=0.3,
                              patience=2,
                              verbose=1,
                              threshold=1e-4,
                              cooldown=2,
                              min_lr=1e-7
                             )

# Early stopping
earlyStopping = EarlyStopping(patient=5)

In [ ]:
optimizer.zero_grad()
data_train_iter = data.DataLoader(dataset['train'], **training_params)

train_loss = []
for epoch in range(1, max_epochs+1):
    batch_loss = []
    pb = ProgressBar(len(data_train_iter), percentPrecision=5, headLabel='Epoch {} '.format(epoch))
    for local_x, _ in data_train_iter:
        if model.onGPU:
            local_x = local_x.cuda()
        
        x_prime = model(local_x)
        loss = criterion(local_x.view(-1, model.No*model.p), x_prime.view(-1, model.No*model.p))
        batch_loss.append(loss.item())
        
        tail_label = 'Loss: {:2.2e}'.format(loss.item())
        if len(train_loss) > 0:
            tail_label += ' ({:2.2e})'.format(loss.item() - train_loss[-1][-1])
        pb.show(len(batch_loss)-1, tail_label=tail_label)
        loss.backward()
        optimizer.step()
        
    aux_loss = 0
    N_batches = 0
    for local_x, _ in data_train_iter:
        if model.onGPU:
            local_x = local_x.cuda()
        loss = criterion(local_x, model(local_x))
        aux_loss += loss.item()
        N_batches += 1
    batch_loss.append(aux_loss/N_batches)
    
    printout = 'Loss: {:2.2e}'.format(batch_loss[-1])
    if len(train_loss) > 0:
        printout += ' - Delta: {:2.2e}'.format(batch_loss[-1] - train_loss[-1][-1])
    if len(train_loss) > 1:
        d2L_de2 = batch_loss[-1] - 2*train_loss[-1][-1] + train_loss[-2][-1]
        printout +=' - Trend: {:2.2e}'.format(d2L_de2)
    print(printout+'\n')
    
    train_loss.append(batch_loss)
    
    if not earlyStopping.check(batch_loss[-1]):
        break

    scheduler.step(batch_loss[-1])

In [ ]:
train_loss = np.array(train_loss)
matplotlib.rcParams.update({'font.size': 16})

f = plt.figure(figsize=(16,8))

train_loss_flat = train_loss.flatten()
x = np.arange(1, train_loss_flat.shape[0]+1) * float(train_loss.shape[0])/train_loss_flat.shape[0]
plt.plot(x, train_loss_flat, '-', alpha=0.2, color='darkorange')

x = np.arange(1, train_loss.shape[0]+1)
plt.plot(x, train_loss[:,-1], 'o--', label='Train', color='darkorange')

plt.xlabel('Epoch')
plt.grid()
plt.legend(loc='best')
plt.yscale('log')
# f.savefig('plots/TrainingHistory_VAE_all-in-one_v'+VAEversion+'.png')

# f = open('TrainingHistory_VAE_all-in-one_v'+VAEversion+'.pkl', 'w')
# pickle.dump(fit_report.history, f, pickle.HIGHEST_PROTOCOL)
# f.close()

# Convergence check

In [ ]:
x = dataset['train'].inputs
model.eval()
x_prime = model(torch.from_numpy(dataset['train'].inputs[:10000])).cpu().detach().numpy()
z = model.encode(torch.from_numpy(dataset['train'].inputs[:10000])).cpu().detach().numpy()

### Latent space

In [ ]:
for i in range(z.shape[1]):
    x_aux = z[:,i]
    f = plt.figure(i)
    r = (np.min(x_aux), np.max(x_aux))
    plt.hist(x_aux, bins=20, range=r, alpha=0.4, density=True)
    plt.xlabel('$z_{{{}}}$'.format(i))
    dnd.append(f)

### Output features

In [ ]:
for i in range(x_prime.shape[2]):
    x_aux = x[:,:,i].flatten()
    xp_aux = x_prime[:,:,i].flatten()
    f = plt.figure(i)
    r = (min(np.min(x_aux), np.min(xp_aux)), max(np.max(x_aux), np.max(xp_aux)))
    plt.hist(x_aux, bins=20, range=r, alpha=0.4, density=True)
    plt.hist(xp_aux, bins=20, range=r, alpha=0.4, density=True)
    plt.xlabel(dataset['train'].feature_names[i])
    dnd.append(f)